## Defining functions inside other functions

In [1]:
def plus_one(number):
    def add_one(number):
        return number+1
    result = add_one(number)
    return result
plus_one(4)

5

## Passing functions as arguments to other functions

In [4]:
def plus_one(number):
    return number + 1

def function_call(number, function):
    # number = 5
    return function(number)

function_call(5, plus_one)

6

## Closure Pattern

In [13]:
def print_message(message):
    def message_sender():
        print(message)
    return message_sender

decorate = print_message("some random message")
decorate()

some random message


## Creating Decorators

In [11]:
def uppercase_decorator(function):
    def wrapper():
        func = function()
        make_uppercase = func.upper()
        return make_uppercase
    
    return wrapper

def say_hi():
    return 'hello there'

decorate = uppercase_decorator(say_hi)
decorate()

'HELLO THERE'

## Using @symbol

In [15]:
@uppercase_decorator
def say_hi():
    return 'hello there'

say_hi()

'HELLO THERE'

In [19]:
def split_string(function):
    def wrapper():
        func = function()
        split_string = func.split()
        return split_string
    
    return wrapper

@split_string
@uppercase_decorator
def say_hi():
    return 'hello there'
    
say_hi()

['HELLO', 'THERE']

## Simple Decorators

In [20]:
def decorator(func):
    def wrapper():
        print(f'this is decorated message before input functions')
        func()
        print(f'this is decorated message after input functions')
    return wrapper

def say_hi():
    print("hi")

say = decorator(say_hi)
say()





this is decorated message before input functions
hi
this is decorated message after input functions


## Syntactic Sugar

In [22]:
@decorator
def say_hi():
    print("say hi")

say_hi()

this is decorated message before input functions
say hi
this is decorated message after input functions


## Reusing Decorators

In [24]:
def do_twice(func):
    def wrapper():
        func()
        func()
    return wrapper

@do_twice
def say_hi():
    print("say hi")

say_hi()

say hi
say hi


## Decorating Functions with Arguments

In [26]:
@do_twice
def greet(name):
    print(f'hello {name}')

greet('world')

TypeError: wrapper() takes 0 positional arguments but 1 was given

In [28]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f'hello {name}')

@do_twice
def say_hi():
    print(f'hi')

greet('world')
say_hi()

hello world
hello world
hi
hi


In [31]:
print(say_hi.__name__)
help(say_hi)

wrapper_do_twice
Help on function wrapper_do_twice in module __main__:

wrapper_do_twice(*args, **kwargs)



In [32]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_hi():
    print(f'hi')

print(say_hi.__name__)
help(say_hi)

say_hi
Help on function say_hi in module __main__:

say_hi()



## Timing Functions

In [33]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [35]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0025 secs


In [36]:
waste_some_time(999)

Finished 'waste_some_time' in 1.6767 secs


## Debugging Code

In [37]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [38]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [41]:
print(make_greeting('Benjamin'))
print(make_greeting('Iggy',age=30))

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'
Howdy Benjamin!
Calling make_greeting('Iggy', age=30)
'make_greeting' returned 'Whoa Iggy! 30 already, you are growing up!'
Whoa Iggy! 30 already, you are growing up!


In [43]:
import math

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [44]:
approximate_e(10)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24
Calling factorial(5)
'factorial' returned 120
Calling factorial(6)
'factorial' returned 720
Calling factorial(7)
'factorial' returned 5040
Calling factorial(8)
'factorial' returned 40320
Calling factorial(9)
'factorial' returned 362880


2.7182815255731922

In [45]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [50]:
print(PLUGINS)
randomly_greet('alice')

{'say_hello': <function say_hello at 0x7f8cb4c5d158>, 'be_awesome': <function be_awesome at 0x7f8cb4c5d0d0>}
Using 'be_awesome'


'Yo alice, together we are the awesomest!'

## Decorating Classes

### @staticmethod

In [63]:
class Calculator:
    @staticmethod
    def addNumbers(x,y):  #self is not as input argument
        return x+y

print(Calculator.addNumbers(2,5))
c = Calculator
print(c.addNumbers(2,3))

7
5


### @property

In [65]:
class C(object):
    def __init__(self):
        self._x = None
 
    def getx(self):
        return self._x
 
    def setx(self, value):
        self._x = value
 
    def delx(self):
        del self._x
 
    x = property(getx, setx, delx, "I'm the 'x' property.")

In [68]:
print(C.x)
c = C
print(c.x)
c.x = 5
print(c.x)

5


In [73]:
class C(object):
    def __init__(self):
        self._x = None
 
    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x
 
    @x.setter
    def x(self, value):
        self._x = value
 
    @x.deleter
    def x(self):
        del self._x

In [74]:
c= C
print(c.x)
c.x = 10
print(c.x)
del c.x
print(c.x)

10


AttributeError: type object 'C' has no attribute 'x'

In [75]:
class C(object):
    def __init__(self):
        self._x = None
 
    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x
 
    @x.setter
    def x(self, value):
        self._x = value
 
    @x.deleter
    def x(self):
        del self._x

In [76]:
c= C
print(c.x)
c.x = 10
print(c.x)
del c.x
print(c.x)

10


AttributeError: type object 'C' has no attribute 'x'

## Nested Decorators

## Decorators with Arguments

In [56]:
import functools

def repeat(num_times):
    def repeat_decorator(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                func(*args, **kwargs)
            return func(*args, **kwargs)
        return wrapper_repeat
    return repeat_decorator

@repeat(num_times=4)
def say_hi(name):
    print(f'hi {name}')

print(say_hi.__name__)
help(say_hi)
say_hi('world')

say_hi
Help on function say_hi in module __main__:

say_hi(name)

hi world
hi world
hi world
hi world
hi world


## Stateful Decorators

In [57]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")


In [58]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [59]:
say_whee()

Call 2 of 'say_whee'
Whee!
